# REFERENCE

4 conditions: bad_neck, bad_back, good_neck, good_back

1. neck_bad = sensor on neck while sitting in bad posture (forward head position)
2. back_bad = sensor on back while sitting in bad posture (forward head position)
3. neck_good = sensor on neck while sitting in good posture
4. back_good = sensor on back while sitting in good posture

# DECODE LOGGED DATA

In [1]:
import os

def decode_logged_file(directory, condition, demo = False):
    '''
    This method is used to read the logged file from the BlueSee application.
    
    Input arguments:
    - directory (String) - name of the directory in which logged files are located.
    - condition (String) - condition of the data collection trial. The 4 conditions are neck_bad, back_bad, neck_goood, and back_good (See above markdown)
    - demo (Boolean) - whether the logged file is for the real-time demo application or not.
    
    Returns:
    - d (Dictionary) - dictionary is returned where the key is the timestamp of the computer and the value is the logged hex data from the sensor
    '''
    d = {}
    for filename in os.listdir(directory):
        file = os.path.join(directory, filename)
        if os.path.isfile(file):
            if condition in filename:
                with open(file, 'r') as f:
                    lines = [line.rstrip() for line in f]
                    for l in lines:
                        if 'received update from characteristic CF54BF43-3D66-4666-8FD3-7DF5788B73C1___E8A68B2A-B616-45C0-B8D0-D9DDF447731E' in l:
                            time = l[:13]
                            data = l[-45:-1]
                            if demo:
                                d[time] = data.replace(" ", "")
                            else:
                                d[time] = (data.replace(" ", ""), filename[:-4]) # filename includes the condition and trial number information (condition_trialnum)
    return d

In [2]:
directory = 'data_collection'

neck_bad = decode_logged_file(directory, 'neck_bad')
back_bad = decode_logged_file(directory, 'back_bad')
neck_good = decode_logged_file(directory, 'neck_good')
back_good = decode_logged_file(directory, 'back_good')

In [4]:
import binascii
import datetime
from struct import unpack

def decode_hex_value(cond_dict, demo = False):
    '''
    This method is used to decode the hex value from the logged data of the sensors. The hex value is deciphered according to the byte information for each element provided by Hocoma.
    
    Input arguments:
    - cond_dict (Dictionary) - the condition dictionary from the output of the decode_logged_file method.
    - demo (Boolean) - whether the logged file is for the real-time demo application or not.
    
    Returns:
    - l (List) - list of tuples is returned that contains the timestamp of the computer, quaternion components (W, X, Y, Z), timestamp of the sensor, sensor status, and label (condition_trialnum)
    '''
    l = []
    for key, value in cond_dict.items():
        if demo:
            pckt = binascii.unhexlify(value)
        else:
            pckt = binascii.unhexlify(value[0])
        # quaternion components (W, X, Y, Z) (32 bits each)
        W = unpack("<f", pckt[0:4])[0] 
        X = unpack("<f", pckt[4:8])[0] 
        Y = unpack("<f", pckt[8:12])[0] 
        Z = unpack("<f", pckt[12:16])[0]
        # sensor timestamp (24 bits)
        TS = int.from_bytes(pckt[16:19], "little") 
        TS = datetime.datetime.fromtimestamp(TS / 1e6) # for microsecond precision
        # sensor status (8 bits)
        S = ''.join(format(byte, '08b')[::-1] for byte in pckt[19:20]) 
        if demo:
            t = (key, (W, X, Y, Z), TS, S)
        else:
            L = value[1] # label
            t = (key, (W, X, Y, Z), TS, S, L)
        l.append(t)
    return l

In [5]:
neck_bad_decode = decode_hex_value(neck_bad)
back_bad_decode = decode_hex_value(back_bad)
neck_good_decode = decode_hex_value(neck_good)
back_good_decode = decode_hex_value(back_good)

# DATAFRAME

In [7]:
import pandas as pd
import numpy as np
pd.options.mode.chained_assignment = None  # default='warn'

def create_dataframe(cond_list, demo = False):
    '''
    This method is used to create a dataframe from the output of the logged file that was decoded.
    
    Input arguments:
    - cond_list (List) - list of tuples of all of the individual elements in the logged data (output of decode_hex_values)
    - demo (Boolean) - whether the logged file is for the real-time demo application or not.
    
    Returns:
    - df (Pandas Dataframe) - dataframe of all of the logged data elements
    '''
    if demo:
        df = pd.DataFrame(data = cond_list, 
                      index = range(0, len(cond_list)),
                      columns = ['Notebook_Timestamp','Quaternion_Orientation_(W, X, Y, Z)','Sensor_Timestamp','Status'])
    else:
        df = pd.DataFrame(data = cond_list, 
                      index = range(0, len(cond_list)),
                      columns = ['Notebook_Timestamp','Quaternion_Orientation_(W, X, Y, Z)','Sensor_Timestamp','Status','Label'])
    df["Notebook_Timestamp"] = pd.to_datetime(df["Notebook_Timestamp"], format='%H:%M:%S.%f')
    return df

In [8]:
# create dataframes for each condition
neck_bad_df = create_dataframe(neck_bad_decode)
back_bad_df = create_dataframe(back_bad_decode)
neck_good_df = create_dataframe(neck_good_decode)
back_good_df = create_dataframe(back_good_decode)

In [11]:
neck_bad_df

,Notebook_Timestamp,"Quaternion_Orientation_(W, X, Y, Z)",Sensor_Timestamp,Status,Label
0,1900-01-01 18:22:15.348,"(0.8028255701065063, -0.2572527229785919, -0.3...",1970-01-01 01:00:02.382962,10010100,neck_bad_8
1,1900-01-01 18:22:15.378,"(0.8034464716911316, -0.25700846314430237, -0....",1970-01-01 01:00:02.402976,10010100,neck_bad_8
2,1900-01-01 18:22:15.393,"(0.8041063547134399, -0.256687730550766, -0.34...",1970-01-01 01:00:02.422991,10010100,neck_bad_8
3,1900-01-01 18:22:15.395,"(0.8048624992370605, -0.25647401809692383, -0....",1970-01-01 01:00:02.443007,10010100,neck_bad_8
4,1900-01-01 18:22:15.408,"(0.8056890964508057, -0.2562243342399597, -0.3...",1970-01-01 01:00:02.463020,10010100,neck_bad_8
...,...,...,...,...,...
49022,1900-01-01 22:12:02.029,"(0.1796412169933319, -0.5263163447380066, -0.0...",1970-01-01 01:00:15.616911,10110000,neck_bad_5
49023,1900-01-01 22:12:02.067,"(0.17984285950660706, -0.526576042175293, -0.0...",1970-01-01 01:00:15.636925,10110000,neck_bad_5
49024,1900-01-01 22:12:02.104,"(0.18042241036891937, -0.5272687077522278, -0....",1970-01-01 01:00:15.676953,10110000,neck_bad_5
49025,1900-01-01 22:12:02.142,"(0.1816667765378952, -0.5280287861824036, -0.0...",1970-01-01 01:00:15.716983,10110000,neck_bad_5


In [9]:
def check_sensor_orientation(df):
    '''
    This method is used to make sure that the sensors are in the correct orientation (i.e., not upside down).
    
    Input arguments:
    - df (Pandas Dataframe) - the dataframe containing the quaternion orientation information to check.
    
    Returns:
    - df (Pandas Dataframe) - filtered if there are any datapoints whose orientation is not correct.
    '''
    df_copy = df.copy(deep=True)
    df_copy['W'] = df['Quaternion_Orientation_(W, X, Y, Z)'].str[0].to_numpy()
    df_copy['X'] = df['Quaternion_Orientation_(W, X, Y, Z)'].str[1].to_numpy()
    df_copy['Y'] = df['Quaternion_Orientation_(W, X, Y, Z)'].str[2].to_numpy()
    df_copy['Z'] = df['Quaternion_Orientation_(W, X, Y, Z)'].str[3].to_numpy()
    
    # upside down orientation was tested with test_up.txt and test_down.txt files in the data_collection folder
    # test_up.txt is the data collected in the correct upright orientation.
    # test_down.txt is the data collected in the upside down orientation.
    df = df_copy[~((df_copy['W'].astype(int) > 0) & (df_copy['X'].astype(int) > 0) & (df_copy['Y'].astype(int) > 0) & (df_copy['Z'].astype(int) < 0))]
    df = df.drop(columns=['W', 'X', 'Y', 'Z'])
    return df

In [10]:
neck_bad_df = check_sensor_orientation(neck_bad_df)
back_bad_df = check_sensor_orientation(back_bad_df)
neck_good_df = check_sensor_orientation(neck_good_df)
back_good_df = check_sensor_orientation(back_good_df)

In [12]:
neck_bad_df

,Notebook_Timestamp,"Quaternion_Orientation_(W, X, Y, Z)",Sensor_Timestamp,Status,Label
0,1900-01-01 18:22:15.348,"(0.8028255701065063, -0.2572527229785919, -0.3...",1970-01-01 01:00:02.382962,10010100,neck_bad_8
1,1900-01-01 18:22:15.378,"(0.8034464716911316, -0.25700846314430237, -0....",1970-01-01 01:00:02.402976,10010100,neck_bad_8
2,1900-01-01 18:22:15.393,"(0.8041063547134399, -0.256687730550766, -0.34...",1970-01-01 01:00:02.422991,10010100,neck_bad_8
3,1900-01-01 18:22:15.395,"(0.8048624992370605, -0.25647401809692383, -0....",1970-01-01 01:00:02.443007,10010100,neck_bad_8
4,1900-01-01 18:22:15.408,"(0.8056890964508057, -0.2562243342399597, -0.3...",1970-01-01 01:00:02.463020,10010100,neck_bad_8
...,...,...,...,...,...
49022,1900-01-01 22:12:02.029,"(0.1796412169933319, -0.5263163447380066, -0.0...",1970-01-01 01:00:15.616911,10110000,neck_bad_5
49023,1900-01-01 22:12:02.067,"(0.17984285950660706, -0.526576042175293, -0.0...",1970-01-01 01:00:15.636925,10110000,neck_bad_5
49024,1900-01-01 22:12:02.104,"(0.18042241036891937, -0.5272687077522278, -0....",1970-01-01 01:00:15.676953,10110000,neck_bad_5
49025,1900-01-01 22:12:02.142,"(0.1816667765378952, -0.5280287861824036, -0.0...",1970-01-01 01:00:15.716983,10110000,neck_bad_5


In [13]:
def synchronize(neck_df, back_df, demo = False, num_trials = 0, is_good = False):
    '''
    This method is used to synchronize the sensor data according to the global notebook and local sensor timestamps.
    
    Input arguments:
    - neck_df (Pandas Dataframe) - dataframe containing the sensor information when it is placed on the neck
    - back_df (Pandas Dataframe) - dataframe containing the sensor information when it is placed on the back
    - demo (Boolean) - whether the logged file is for the real-time demo application or not.
    - num_trials (Integer) - number of trials of data collection to iterate over
    - is_good (Boolean) - whether the condition is the Good Posture condition.
    
    Returns:
    - df (Pandas Dataframe) - synchronized dataframe of the neck_df and back_df
    '''
    
    if demo:
        
        neck_df, back_df = synchronize_notebook_timestamps(neck_df, back_df)
        df = synchronize_sensor_timestamps(neck_df, back_df)
        
    else:
        
        # iterate over trials
        trials = list(range(1, num_trials + 1))

        is_good_df = []
        all_df = []

        for t in trials:
            
            # get the filtered dataframe for the trial of interest
            if t < 10:
                neck_df_t = neck_df[neck_df['Label'].str.slice(start=-2) == "_{}".format(t)]
                back_df_t = back_df[back_df['Label'].str.slice(start=-2) == "_{}".format(t)]
            else:
                neck_df_t = neck_df[neck_df['Label'].str.slice(start=-2) == str(t)]
                back_df_t = back_df[back_df['Label'].str.slice(start=-2) == str(t)]

            neck_df_t.reset_index(drop=True, inplace=True)
            back_df_t.reset_index(drop=True, inplace=True)
            
            # first synchronize the notebook timestamps
            neck_df_t, back_df_t = synchronize_notebook_timestamps(neck_df_t, back_df_t)

            # then synchronize the sensor timestamps 
            df = synchronize_sensor_timestamps(neck_df_t, back_df_t)

            # in the Good Posture condition, not much movement was made. 
            # Therefore, to get more variation of data, data from several trials are combined
            if is_good:
                if (df.shape[0] >= 250) and (len(is_good_df) != 4): # make sure a trial has 1000 datapoints when concatenated
                    df = df.head(250)
                    is_good_df.append(df)
                    if len(is_good_df) == 4:
                        df = concat_dataframes(is_good_df, sort_col='Notebook_Timestamp_Neck')
                        df['Label_Neck'] = "neck_good_{}".format(t)
                        df['Label_Back'] = "back_good_{}".format(t)
                        df = df.head(1000) # one trial has 1000 datapoints
                        all_df.append(df)
                        is_good_df.clear()
                else:
                    print("trial number {} has insufficient number of datapoints: {}".format(t, df.shape[0]))
            
            # More movement was made during data collection of FHP. Therefore concatenating data among trials was not necessary
            if not is_good:
                if df.shape[0] >= 1000:
                    df = df.head(1000) # one trial has 1000 datapoints
                    all_df.append(df)
                else:
                    print("trial number {} has insufficient number of datapoints: {}".format(t, df.shape[0]))
            
        df = concat_dataframes(all_df, sort_col='Notebook_Timestamp_Neck')

    return df

def synchronize_notebook_timestamps(neck_df, back_df):
    '''
    This method is used to match the first notebook timestamp of the two sensor logged data of the neck and back.
    The dataframes should be the same condition and trial.
    
    Input arguments:
    - neck_df (Pandas Dataframe) - dataframe containing the sensor information when it is placed on the neck
    - back_df (Pandas Dataframe) - dataframe containing the sensor information when it is placed on the back
    
    Returns:
    - neck_df (Pandas Dataframe) - dataframe that is filtered to match the first notebook timestamp
    - back_df (Pandas Dataframe) - dataframe that is filtered to match the first notebook timestamp
    '''
    
    # first notebook timestamp of each dataframe
    neck_nb_ts = neck_df['Notebook_Timestamp'].iloc[0]
    back_nb_ts = back_df['Notebook_Timestamp'].iloc[0]
    
    # if the notebook timestamp starts earlier for the neck dataframe
    if (neck_nb_ts < back_nb_ts):
        # the neck dataframe starting index is delayed
        start_index = neck_df.index[[(abs(neck_df.iloc[:, 0]-back_nb_ts)).idxmin()]].item()
        neck_df = neck_df[neck_df.index >= start_index]
    else:
        start_index = back_df.index[[(abs(back_df.iloc[:, 0]-neck_nb_ts)).idxmin()]].item()
        back_df = back_df[back_df.index >= start_index]
        
    return neck_df, back_df

def synchronize_sensor_timestamps(neck_df, back_df):
    '''
    This method is used to synchronize the sensor timestamps of the two sensor logged data of the neck and back.
    The dataframes should be the same condition and trial, and already matched to the first notebook timestamp.
    
    Input arguments:
    - neck_df (Pandas Dataframe) - dataframe containing the sensor information when it is placed on the neck
    - back_df (Pandas Dataframe) - dataframe containing the sensor information when it is placed on the back
    
    Results:
    - df (Pandas Dataframe) - merged dataframe of the sensor_neck and sensor_back data by synchronizing the sensor timestamps
    '''
    
    neck_s_ts1 = neck_df['Sensor_Timestamp'].iloc[0]
    neck_s_ts2 = neck_df['Sensor_Timestamp'].iloc[1]
    back_s_ts1 = back_df['Sensor_Timestamp'].iloc[0]
    back_s_ts2 = back_df['Sensor_Timestamp'].iloc[1]

    # get the sensor timestamp difference 
    neck_s_ts_diff = neck_s_ts2 - neck_s_ts1
    back_s_ts_diff = back_s_ts2 - back_s_ts1

    neck_df.reset_index(drop=True, inplace=True)
    back_df.reset_index(drop=True, inplace=True)

    # make sure timestamp is increasing at a constant rate with the sensor timestamp difference
    for i in range(1, neck_df.shape[0]):
        prev = neck_df['Sensor_Timestamp'].iloc[i-1]
        if (neck_df['Sensor_Timestamp'].iloc[i] < prev):
            neck_df.loc[i, 'Sensor_Timestamp'] = prev + neck_s_ts_diff

    for i in range(1, back_df.shape[0]):
        prev = back_df['Sensor_Timestamp'].iloc[i-1]
        if (back_df['Sensor_Timestamp'].iloc[i] < prev):
            back_df.loc[i, 'Sensor_Timestamp'] = prev + back_s_ts_diff

    # if the sensor timestamp starts earlier for the neck dataframe
    if (neck_s_ts1 < back_s_ts1):
        # the sensor timestamps of the neck are shifted by the difference between the sensor timestamps of the neck and back dataframes
        s_ts_diff = back_s_ts1 - neck_s_ts1
        neck_df['Sensor_Timestamp'] = neck_df['Sensor_Timestamp'] + s_ts_diff

    else:
        s_ts_diff = neck_s_ts1 - back_s_ts1
        back_df['Sensor_Timestamp'] = back_df['Sensor_Timestamp'] + s_ts_diff

    # set the starting time of the sensor timestamp to 0
    neck_s_ts = neck_df['Sensor_Timestamp'].iloc[0]
    back_s_ts = back_df['Sensor_Timestamp'].iloc[0]
    neck_df['Sensor_Timestamp'] = neck_df['Sensor_Timestamp'] - neck_s_ts
    back_df['Sensor_Timestamp'] = back_df['Sensor_Timestamp'] - back_s_ts
    
    # make sure that the dataframes for the neck and back are of the same shape
    if (neck_df.shape[0] < back_df.shape[0]):
        back_df = back_df[back_df.index < neck_df.shape[0]]
    else:
        neck_df = neck_df[neck_df.index < back_df.shape[0]]

    # merge data on the synchronized sensor timestamps
    back_df = back_df.set_index('Sensor_Timestamp').reindex(neck_df.set_index('Sensor_Timestamp').index, method='nearest').reset_index()
    df = pd.merge(neck_df, back_df, on='Sensor_Timestamp', suffixes=('_Neck', '_Back'))
    df = format_timestamps(df, ['Notebook_Timestamp_Neck', 'Notebook_Timestamp_Back'])
    
    return df

def concat_dataframes(cond_list, sort_col):
    '''
    This method is used to concatenate dataframes in a list.
    
    Input arguments:
    - cond_list (List) - list of dataframes to concatenate
    - sort_col (String) - column of dataframe to sort the data by
    
    Results:
    - df (Pandas Dataframe) - concatenated dataframe
    '''
    df = pd.concat(cond_list)
    df = df.sort_values(by=[sort_col])
    df.reset_index(drop=True, inplace=True)
    return df

def format_timestamps(df, col_name_list):
    '''
    This method is used to format the timestamp data.
    
    Input arguments:
    - df (Pandas Dataframe) - dataframe that contains a timestamp column to format
    - col_name_list (List) - list of columns to format the timestamps
    
    - df (Pandas Dataframe) - dataframe whose timestamp columns are formatted
    '''
    for col in col_name_list:
        df[col] = df[col].dt.strftime("%H:%M:%S.%f")
    return df

In [14]:
# synchronize sensors for each condition
bad_df_sync = synchronize(neck_bad_df, back_bad_df, num_trials = 8)
good_df_sync = synchronize(neck_good_df, back_good_df, num_trials = 22, is_good = True)

trial number 1 has insufficient number of datapoints: 402
trial number 2 has insufficient number of datapoints: 253
trial number 3 has insufficient number of datapoints: 225
trial number 1 has insufficient number of datapoints: 172
trial number 2 has insufficient number of datapoints: 114


In [15]:
bad_df_sync

,Notebook_Timestamp_Neck,"Quaternion_Orientation_(W, X, Y, Z)_Neck",Sensor_Timestamp,Status_Neck,Label_Neck,Notebook_Timestamp_Back,"Quaternion_Orientation_(W, X, Y, Z)_Back",Status_Back,Label_Back
0,17:49:27.338000,"(0.8820686340332031, 0.06307948380708694, -0.4...",0 days 00:00:00,00000000,neck_bad_6,17:49:27.331000,"(0.21272346377372742, -0.45321473479270935, 0....",00100000,back_bad_6
1,17:49:27.339000,"(0.8830152750015259, 0.06327345967292786, -0.4...",0 days 00:00:00.020011,00000000,neck_bad_6,17:49:27.345000,"(0.2131420373916626, -0.45195308327674866, 0.0...",00100000,back_bad_6
2,17:49:27.353000,"(0.8840718865394592, 0.06323843449354172, -0.4...",0 days 00:00:00.040027,00000000,neck_bad_6,17:49:27.375000,"(0.21375982463359833, -0.450428307056427, 0.08...",00100000,back_bad_6
3,17:49:27.383000,"(0.8850886821746826, 0.06266017258167267, -0.4...",0 days 00:00:00.060039,00000000,neck_bad_6,17:49:27.376000,"(0.21421855688095093, -0.4489484131336212, 0.0...",00100000,back_bad_6
4,17:49:27.398000,"(0.8860152363777161, 0.0619862899184227, -0.41...",0 days 00:00:00.080054,00000000,neck_bad_6,17:49:27.390000,"(0.21453069150447845, -0.4473279118537903, 0.0...",00100000,back_bad_6
...,...,...,...,...,...,...,...,...,...
4995,22:08:30.412000,"(0.7267450094223022, -0.3523421585559845, -0.1...",0 days 00:00:20.874670,10110000,neck_bad_5,22:08:15.282000,"(0.127139151096344, -0.4554588496685028, 0.047...",00000100,back_bad_5
4996,22:08:30.450000,"(0.7331184148788452, -0.3578275144100189, -0.1...",0 days 00:00:20.894684,10110000,neck_bad_5,22:08:15.318000,"(0.12679548561573029, -0.4563538730144501, 0.0...",00000100,back_bad_5
4997,22:08:30.487000,"(0.736203134059906, -0.36073264479637146, -0.0...",0 days 00:00:20.914698,10110000,neck_bad_5,22:08:15.355000,"(0.12662485241889954, -0.4575359523296356, 0.0...",00000100,back_bad_5
4998,22:08:30.524000,"(0.7421345710754395, -0.3662568926811218, -0.0...",0 days 00:00:20.934712,10110000,neck_bad_5,22:08:15.356000,"(0.1265890747308731, -0.45894962549209595, 0.0...",00000100,back_bad_5


In [16]:
df = concat_dataframes([bad_df_sync, good_df_sync], sort_col='Notebook_Timestamp_Neck')
df

,Notebook_Timestamp_Neck,"Quaternion_Orientation_(W, X, Y, Z)_Neck",Sensor_Timestamp,Status_Neck,Label_Neck,Notebook_Timestamp_Back,"Quaternion_Orientation_(W, X, Y, Z)_Back",Status_Back,Label_Back
0,01:56:20.003000,"(0.8322255611419678, -0.04902138561010361, -0....",0 days 00:00:00,00100000,neck_good_10,01:56:19.997000,"(0.7108583450317383, -0.2387552261352539, -0.6...",00100000,back_good_10
1,01:56:20.033000,"(0.8322974443435669, -0.04904297739267349, -0....",0 days 00:00:00.020012,00100000,neck_good_10,01:56:20.011000,"(0.7109321355819702, -0.23853595554828644, -0....",00100000,back_good_10
2,01:56:20.034000,"(0.8323510885238647, -0.049045007675886154, -0...",0 days 00:00:00.040023,00100000,neck_good_10,01:56:20.012000,"(0.7110407948493958, -0.23825488984584808, -0....",00100000,back_good_10
3,01:56:20.063000,"(0.8324238061904907, -0.04885202646255493, -0....",0 days 00:00:00.060031,00100000,neck_good_10,01:56:20.013000,"(0.7111358046531677, -0.23808898031711578, -0....",00100000,back_good_10
4,01:56:20.078000,"(0.8325163722038269, -0.0486564114689827, -0.5...",0 days 00:00:00.080042,00100000,neck_good_10,01:56:20.027000,"(0.7111834287643433, -0.2379899024963379, -0.6...",00100000,back_good_10
...,...,...,...,...,...,...,...,...,...
9995,22:08:30.412000,"(0.7267450094223022, -0.3523421585559845, -0.1...",0 days 00:00:20.874670,10110000,neck_bad_5,22:08:15.282000,"(0.127139151096344, -0.4554588496685028, 0.047...",00000100,back_bad_5
9996,22:08:30.450000,"(0.7331184148788452, -0.3578275144100189, -0.1...",0 days 00:00:20.894684,10110000,neck_bad_5,22:08:15.318000,"(0.12679548561573029, -0.4563538730144501, 0.0...",00000100,back_bad_5
9997,22:08:30.487000,"(0.736203134059906, -0.36073264479637146, -0.0...",0 days 00:00:20.914698,10110000,neck_bad_5,22:08:15.355000,"(0.12662485241889954, -0.4575359523296356, 0.0...",00000100,back_bad_5
9998,22:08:30.524000,"(0.7421345710754395, -0.3662568926811218, -0.0...",0 days 00:00:20.934712,10110000,neck_bad_5,22:08:15.356000,"(0.1265890747308731, -0.45894962549209595, 0.0...",00000100,back_bad_5


In [17]:
# save final data as .csv format for future use
csv_filename = 'read_file_bluesee_1_df_final.csv'
df.to_csv(csv_filename, index=False)